# World Bank - Population Summarizing
Reduced '.csv' files from WB_data Notebook

In [36]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests
import matplotlib.pyplot as plt
from matplotlib import cbook
from scipy.stats import linregress
import scipy.stats as st
import numpy as np
import time


# Import the Geoapify API key
#from api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
#from citipy import citipy

In [37]:
# WB data set
WB_africadata_2000 = Path("Resources\WB_africadata_2000.csv")

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hebir\AppData\Local\Temp\ipykernel_18780\2038409107.py:2: SyntaxWarning: invalid escape sequence '\W'
  WB_africadata_2000 = Path("Resources\WB_africadata_2000.csv")


In [6]:
# Read data file with the Pandas library
# encoding?, i.e encoding="ISO-8859-1"
WB_africadata_2000_df = pd.read_csv(WB_africadata_2000)

FileNotFoundError: [Errno 2] No such file or directory: 'Resources\\WB_africadata_2000.csv'

In [ ]:
WB_africadata_2000_df

### World Bank Data Source Break-down
###The 'WB_africadata_2000.csv' created in the WB_data notebook will be used to summarize the Foreign Aid and different indicators data.

###To help with the anlysis of the data, the data was put in bins of 5 years from 2000 to 2020, and a last bin for 2021 to 2023.

In [ ]:
WB_africadata_2000_df2 = WB_africadata_2000_df

# Slicing of the Data
###In order to analyze the data and to understand the impact of the Foreign Aid with regards to poverty rate, literacy and mortality, the data was sliced by the "series_id". The deifinition of the diferent "series_id" values (indicators) is described in a second file downloaded from the Nasdaq API and saved as 'WB_metadata_df'.

###To facilitate/expedite the search and filtering of the relevant a function was created to search the text strings : "key_word". This function is case sensitive and it's used is combined with the review of the search output selection to identify the indicators that best match our work objective.

In [9]:
# WB id series definition data set
WB_metadata = Path("Resources/WB_METADATA_f7ce7fba293ccc6eb39cdf15fb097982.csv")

In [10]:
# Read data file with the Pandas library
WB_metadata_df = pd.read_csv(WB_metadata)

FileNotFoundError: [Errno 2] No such file or directory: 'Resources\\WB_METADATA_f7ce7fba293ccc6eb39cdf15fb097982.csv'

In [ ]:
WB_metadata_df

In [ ]:
# This function will search within a text string for a given 'key word'. It is case sensitive

def key_word(df, search_col, word_txt, result_col):
    i=0   
    result_ls = []
    for item in df[search_col]:
        text_ls = [x.strip() for x in df[search_col][i].split()]
        if word_txt in text_ls:
            result_ls.append(df[result_col][i])
        i+=1
    if i >= df[search_col].count():
        if len(result_ls) > 0:
            return result_ls
        else:
            return print("Keyword Not found")

In [ ]:
population_search = key_word(WB_metadata_df, 'name', 'population', 'series_id')

In [ ]:
#population_search

In [ ]:
population_search2 = WB_metadata_df[WB_metadata_df['series_id'].isin(population_search)]

In [ ]:
#population_search2

# Selection of Indicators for Analysis

###Indicators are selected out of the search result and used to create reduc ed data sets. In the case of the Population indicator, we selected six types of indicators : Rural and Urban population quantity, Rural and Urban population percent of total population, and Rural and Urban population annual growth % per year.

###In order to organize the data, we prepared 6 data sets. One for each indicator.

In [ ]:
#SP.URB.TOTL:Urban population, SP.URB.TOTL.IN.ZS:Urban population (% of total population), SP.URB.GROW:Urban population growth (annual %)

#SP.RUR.TOTL:Rural population, SP.RUR.TOTL.ZS:Rural population (% of total population), SP.RUR.TOTL.ZG:Rural population growth (annual %),

#population_search_choiceURBT = ['SP.URB.TOTL']
#population_search_choiceRURT = ['SP.RUR.TOTL']
#population_search_choiceURBP = ['SP.URB.TOTL.IN.ZS']
#population_search_choiceRURP = ['SP.RUR.TOTL.ZS']
#population_search_choiceURBG = ['SP.URB.GROW']
#population_search_choiceRURG = ['SP.RUR.TOTL.ZG']

## Total Urban Population

In [ ]:
WB_pop_URBT_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == 'SP.URB.TOTL']
#WB_pop_URBT_df

In [ ]:
reduc_pop_URBT_df = WB_pop_URBT_df.reset_index()
reduc_pop_URBT_df.drop(columns=['index','series_id'], inplace=True)
reduc_pop_URBT_df

In [ ]:
URBT_table = reduc_pop_URBT_df.pivot(index='year', columns='country_name', values='value')
#URBT_table

## Total Rural Population

In [ ]:
WB_pop_RURT_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == 'SP.RUR.TOTL']
#WB_pop_RURT_df

In [ ]:
reduc_pop_RURT_df = WB_pop_RURT_df.reset_index()
reduc_pop_RURT_df.drop(columns=['index','series_id'], inplace=True)
#reduc_pop_RURT_df

In [ ]:
RURT_table = reduc_pop_RURT_df.pivot(index='year', columns='country_name', values='value')
#RURT_table

# Total Population

In [ ]:
TOT_table = RURT_table + URBT_table
TOT_table

In [ ]:
data2 = {
    '2000 Total Population (millions)': (TOT_table.iloc[0, 0:31]/1_000_000),
    '2023 Total Population (millions)': (TOT_table.iloc[-1, 0:31]/1_000_000)   
}
summary_table2 = pd.DataFrame(data2)
summary_table2


In [ ]:
# Create a sample DataFrame
data = {
    '2000': (summary_table2['2000 Total Population (millions)']),
    '2023': (summary_table2['2023 Total Population (millions)'])
}
df = pd.DataFrame(data)

# Plot overlapping histograms
df.plot.hist(alpha=0.5, bins=10)
plt.title ("Total Population distribution 2000 vs 2023")
plt.xlabel('Total Population (millions)')
plt.ylabel('Country Counts')
plt.show()

# Add code to create figure output

In [ ]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
summary_table2.to_csv("Resources\WB_population_summary.csv", encoding="utf-8", index=False)

In [ ]:
# Sample data
data1 = {'Category': ['A', 'A', 'A', 'B', 'B', 'B'], 'Value': [1, 2, 3, 4, 5, 6]}
data2 = {'Category': ['A', 'A', 'A', 'B', 'B', 'B'], 'Value': [2, 3, 4, 5, 6, 7]}

# Create DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Combine DataFrames
df1['Dataset'] = 'Dataset 1'
df2['Dataset'] = 'Dataset 2'
combined_df = pd.concat([df1, df2])

# Plot box plots
plt.figure(figsize=(10, 6))
combined_df.boxplot(by='Category', column='Value', grid=False)
plt.title('Box Plots of Two Datasets')
plt.suptitle('')  # Suppress the default title
plt.xlabel('Category')
plt.ylabel('Value')
plt.show()

In [ ]:
combined_df

In [ ]:
# My code - not working - DO NOT RUN!
# Sample data
data1 = {'Category': (WB_africadata_2000_df2['country_name'].unique()), 'Value': (summary_table2['2000 Total Population (millions)'])}
data2 = {'Category': (WB_africadata_2000_df2['country_name'].unique()), 'Value': (summary_table2['2023 Total Population (millions)'])}

# Create DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Combine DataFrames
df1['Dataset'] = '2000'
df2['Dataset'] = '2023'
combined_df = pd.concat([df1, df2])

# Plot box plots
plt.figure(figsize=(10, 6))
combined_df.boxplot(by='Category', column='Value', grid=False)
plt.title('Box Plots of Two Datasets')
plt.suptitle('')  # Suppress the default title
plt.xlabel('Category')
plt.ylabel('Value')
plt.show()

In [ ]:
# My code - not working - DO NOT RUN!
data_2000 = {
    '2000': (summary_table2['2000 Total Population (millions)']),
    '2023': (summary_table2['2023 Total Population (millions)'])
}
df = pd.DataFrame(data2)

fig, (ax1) = plt.subplots()

# single boxplot call
summary_table2.boxplot(by='country_name', column)
ax1.boxplot(data2, patch_artist=True, boxprops={'facecolor': 'bisque'})
ax1.set_xticklabels=(['2000', '2023']),

plt.show()


In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
map_plot_humidity

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 10,
    scale = 1.0,
    color = "City",
    hover_cols = (['Lng','Lat','City', 'Country','Hotel Name'])
   )

# Display the map
map_plot_hotel

In [35]:
TOT_table.head

NameError: name 'TOT_table' is not defined